In [1]:
import pandas as pd

from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

import os

import gradio as gr

c:\Users\rosha\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")


In [3]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("Data.txt")
docs=loader.load()
docs

[Document(page_content='What is Amazon Pay Later?\nAmazon Pay Later is the hassle-free way to get instant credit, via a completely digital process, for purchases using EMI on Amazon.in. You have to complete the one-time setup process, which should not take more than 2 minutes and does not require you to provide credit card details.\n\nOnce the setup is complete, you can avail Amazon Pay Later payment option during checkout on Amazon.in, and pay later next month or over EMIs ranging from 3 to 12 months. You can easily track your purchases, repayments, and limits history from a simplified dashboard for this payment mode. Amazon Pay Later is offered to you by Amazon Finance India Private Limited ("Amazon") in partnership with one of its third-party lending partners - axio or IDFC FIRST Bank.\n\nWhat are the key benefits of Amazon Pay Later?\nGet instant decision on your credit limit by the lender\nCredit card details not required\nNo processing or cancellation fee\nNo pre-closure charges\

In [4]:
embeddings = embed
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs)
vectors = FAISS.from_documents(final_documents,embeddings)

In [5]:
from sqlalchemy.sql.expression import true

llm=llm

conversational_memory=ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectors.as_retriever()
)

In [6]:
tools=[
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering questions and give a precise answer by getting more information about the question asked'
        )
        
    )
]

In [7]:
agent=initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=False,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

c:\Users\rosha\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
state = {'greeted': False}

with gr.Blocks(title="Ava Payment Query Chatbot") as demo:
  
  gr.Markdown("# Ava Payment Query Chatbot")
  
  
  chatbot = gr.Chatbot()
  msg = gr.components.Textbox()
  clear = gr.ClearButton([msg, chatbot])
  
  def respond(message, chat_history):
    
    bot_message = agent.run(message)
    print(chat_history)
    chat_history.append((message, bot_message))
    return "", chat_history
  
  def greet(chat_history):
        if not state['greeted']:
            chat_history.append(("", "Hi! \nI am Ava - Payment Query Chatbot of Amazon.\nHow can I help you today?"))
            state['greeted'] = True
        return chat_history
  
  msg.focus(greet,[chatbot],[chatbot])
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://029e1983bcbb1cdec5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\rosha\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[['', 'Hi! \nI am Ava - Payment Query Chatbot of Amazon.\nHow can I help you today?']]
[['', 'Hi! \nI am Ava - Payment Query Chatbot of Amazon.\nHow can I help you today?'], ['What can be the reason for payment failures?', 'Payment failures can occur due to incorrect information entered, such as a wrong credit/debit card number, expiration date, CVV, billing address, or phone number.']]
